In [1]:
import os
import igraph as ig
import numpy as np
import netwulf as nw
import networkx as nx
import leidenalg as la
import louvain as lo
if os.getcwd()[-1] in '0123456798':
    path_parent = os.path.dirname(os.getcwd())
    os.chdir(path_parent)

C:\Users\toell\AppData\Local\Temp\ipykernel_32200\1643069850.py:7: DeprecationWarning: This package has been superseded by the `leidenalg` package and will no longer be maintained. Please upgrade to the `leidenalg` package.
  import louvain as lo


In [2]:
def modularity(G, communities):
    m = G.number_of_edges()
    q = 0.0
    for c in communities:
        lc = G.subgraph(c).number_of_edges()
        kc = sum(G.degree(n) for n in c)
        q += lc/m - (kc/(2*m))**2
    return q

In [3]:
stylized_network, config, G = nw.load("BigData/Graph.json")
Graph = ig.Graph.from_networkx(G)

In [4]:
partition = la.find_partition(Graph, la.ModularityVertexPartition)

In [ ]:
ig.plot(partition)

In [ ]:
partition_nw = partition.to_networkx()
nw.visualize(partition_nw)

In [ ]:
print(modularity())